# Managing your content

As an organization matures and expands its GIS, users add items of various types and properties with varying relationships to one another. Any administrator must regularly manage items contained in various groups and owned by various users. In this section we demonstrate how to work with individual items in a GIS. This guide shows how to retrieve item properties, delete an existing item, and how to examine relationships between items.


<a id="properties-of-an-item"></a>
## Properties of an item
An `Item` in your GIS is rich with multiple properties. You can access them as properties on the [`Item`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#item) object;

In [ ]:
#connect to GIS
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')

In [ ]:
#access an Item (You will need an item in your own GIS here !)
waze_item = gis.content.get('3b5370588fea42de969c7b461787eced')
waze_item

In [ ]:
# item id
waze_item.id

In [ ]:
# title
waze_item.title

In [ ]:
# tags
waze_item.tags

In [ ]:
waze_item.__dict__

<a id="updating-item-properties"></a>
### Updating item properties
You can update any of the `Item`s properties using the [`update()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.update) method. It accepts parameters similar to [`add()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.add) method.

In [ ]:
# update the tags
waze_item.update(item_properties={'tags':'waze, dashboards, devsummit'})

In [ ]:
waze_item.tags

In [ ]:
# updating thumbnail
waze_item.update(thumbnail=r'.\img\wazeimg2.png')

In [ ]:
waze_item

<a id="downloading-your-items"></a>
## Downloading your items
You can download various components of your items, such as the thumbnail, data, or metadata. Downloading assists you in the process of archiving content for a GIS your organization is retiring, or for publishing and migrating content from one GIS to another.

### Download item data
The [`get_data()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.get_data) function reads the binary or text data associated with an item and returns it as a byte array. 
 * If the text data is JSON, it converts the data to a Python dictionary. If it is another text format, it returns the file as a string.
 * If data is not text, binary files are returned along with the path to where the data is downloaded.

In [ ]:
csv_item = gis.content.get('75acdaba698c44fd911a8dfe91eb757c')
csv_item

The data for a csv item is the csv file itself. It's downloaded to your default temporary directory

In [ ]:
waze_item.get_data()

### Download thumbnail
Finally, you can download the thumbnail of an item using the `download_thumbnail()` method.

In [ ]:
waze_item.download_thumbnail(save_folder= r'.\img\temp.png')

## Deleting content
You can delete any item using the `delete()` method of the `Item` object. When deleting a hosted layer, this operation also deletes the hosted services powering the item.

In [ ]:
waze_item

In [ ]:
#waze_item.delete()

#### Delete protection
You can protect items from getting deleted inadvertently. Use the `protect()` of the `Item` object for this. 

In [ ]:
# let us protect the ports item we accessed earlier
waze_item.protect(enable = True)

In [ ]:
# attempting to delete will return an error
waze_item.delete()

As expected an exception is raised. You can disable delete protection by setting the `enable` parameter to `False`.

<a id ="finding-relationships-between-items"></a>
## Finding relationships between items
You can add many [types of items](http://enterprise.arcgis.com/en/portal/latest/use/supported-items.htm) to your web GIS. Depending upon the type of item, you can then perform different operations on that item. Once an item is added to the GIS, it  seldom exists in isolation but often relates to other items. You might [publish](http://enterprise.arcgis.com/en/portal/latest/use/publish-features.htm) new items from other items to create a new service, or create a new service as a result of a geoprocessing operation on a source item. You may also add one or more layer items to compose a web map or web scene item. Whenever you perform such operations, you establish a relationship between the items. A GIS supports different [relationship types](https://developers.arcgis.com/rest/users-groups-and-items/relationship-types.htm) depending upon the items involved. These relationships represent how items in your GIS are connected to one another. They also convey the impact removing an item from your GIS may have on other items. See the [Relationships between web services and portal items](https://enterprise.arcgis.com/en/server/latest/administer/linux/relationships-between-web-services-and-portal-items.htm) article for more information.

The `Item` class has 3 methods that allow you to determine the relationships to and from an item:
 * [`dependent_upon()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.dependent_upon)
 * [`dependent_to()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.dependent_to)
 * [`related_items()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.related_items)

Let us observe how the ports csv item and the feature layer published from this item share a relationship

> **NOTE:** The Relationship Type functionality is currently evolving within Enterprise and ArcGIS Online implementations. Some items do not currently return all relationships they have between other items.  

In [ ]:
waze_item = gis.content.get('3b5370588fea42de969c7b461787eced')
waze_item

Since we know this feature layer item is published from the csv, we can specify the `relationship_type` parameter as `Service2Data`. The direction of relationship would be `forward` as the current item is a service and we are looking for the original data used to publish it.

In [ ]:
waze_item.related_items('Service2Data', 'forward')

Be careful, some functions are only available on ArcGIS Enterprise

https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.dependent_to


## Clone items


One request the Python API team has heard repeatedly from Web GIS administrators: What's the best way to move my content from a development Enterprise or organization to staging to production?

This Guide provides one possible roadmap for how to get content from one Web GIS to another - a template for demonstrating basic software concepts when transferring content so the workflow can be modified for other types of content and tailored for an administrator's particular need. So, first question to answer is: What does it mean to `clone`? Merriam-Webster defines [`clone`](https://www.merriam-webster.com/dictionary/cloning) as the action of _"propagat[ing] a clone from."_ (Didn't I hear my 2nd grade teacher - and every teacher after that for that matter - say it's unhelpful to use the word you are defining in its definition?...) Diving further into the definition of a _"clone"_, it reads _"the aggregate of genetically identical cells or organisms asexually produced by or from a single progenitor cell or organism."_ Okay, not exactly the context we're working in when discussing a Web GIS. Further along the definition we get to something more along the lines of what we're thinking: _"one that appears to be a copy of an original form."_ BINGO! 

Leveraging the Python API to clone content from one Web GIS to another can be thought of as _"propagating content that appears to be a copy of the original content."_. Given all the different content and item types, possible Enterprise configurations, potential ArcGIS Online Organization arrangements, security considerations, and item dependencies the [`clone_items()`]((https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.clone_items)) function aims to _propagate_ an exact duplicate of one item with all its uses and functionality from an original portal to a second portal, but there may be circumstances that lead the clone to come close but not quite produce an identical copy. In addition, the variety of means for publishing items creates varying possibilities for supported, related, or source items created during the publishing process. The relationship between any supported, related, or source items must be considered when cloning an item. For example, the overwrite functionality will not be available on cloned hosted feature layer items, because they do not have a source document from which they were published. The FAQ [here](https://doc.arcgis.com/en/arcgis-online/reference/faq.htm#DELETE_SRC) provides important information regarding source items in ArcGIS Online. However, given our education on `cloning`, we now know that cloning an item produces a second item which appears - but may not be identical to the original. It's important to temper expectations when thinking about the Python API `clone_items()` function. There may be times items will not migrate as expected. 

When talking about cloning items, let's establish some basic terminology.  Cloning can refer to all kinds of transfer, from ArcGIS Enterprise to ArcGIS Online, Enterprise to Enterprise, or ArcGIS Online to Enterprise. (The GIS Community less frequently discusses the need to clone between two distinct ArcGIS Online Organizations, but that is possible as well.) Rather than tangling ourselves up when trying to distinguish between these portal types while writing about them, we'll refer to the portal or organization where the content originates as the _source_ regardless of portal type. We'll consider the portal or organization to which we'll clone content as the _target_. Onward we go!

This document illustrates the workflow of gathering _source_ items using a source administrator account, and cloning them into a _target_ administrator account.  More specifically, this guide walks through an approach of cloning data services known as [`hosted feature layers`](https://doc.arcgis.com/en/arcgis-online/manage-data/hosted-web-layers.htm) that are used as building blocks for other Web GIS items, namely [`web maps`](https://developers.arcgis.com/documentation/core-concepts/web-maps/). `Web Maps` consume these data services as operational layer building blocks. We'll then clone an [`ArcGIS Dashboard`](https://doc.arcgis.com/en/dashboards/get-started/what-is-a-dashboard.htm) and a [`Web Mapping Application`](https://doc.arcgis.com/en/arcgis-online/create-maps/create-map-apps.htm) that each consume a `web map` as part of their definition. For information on each of these item types, please click the hyperlinks for each type.

This is an administrator workflow. Using administrator accounts guarantees all necessary privileges within the _source_ to access the item and any dependencies and within the _target_ to create the services and items. The target administrator subsequently can create user accounts and reassign item ownership and/or group membership according to need. This might also be the simplest workflow when the portal has users managed by an external identity store.

> **NOTE:** Cloning relies on the sharing model to determine the items any particular [`user`](https://develoipers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user) can clone. If a user can access an item, that user can clone it. However, that user can't create anything in the target portal they don't have the appropriate [`privileges`](https://enterprise.arcgis.com/en/portal/latest/administer/windows/roles.htm#ESRI_SECTION1_4FF9051EFB814C249AB26B0ACFF7C79F) to create. 

Any content in any portal must be owned by a [`user`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user), so at least one _user_ must exist in the target. If you don't plan on using the administrator accounts in a workflow such as this, you may want to investigate the [`Clone Portal users, groups and content`](../sample-notebooks/clone-portal-users-groups-and-content/#Copy-Users) sample notebook on how to define functions that can copy `user` objects and then clone content using those accounts. 

As originally written, Python API developers designed the `clone_items()` function for transferring the following item types:
 * Hosted Web Applications built with Web AppBuilder or shared using Configurable App Templates
 * Web Maps
 * Hosted Feature Layers
 * Hosted Feature Layer Views
 * Feature Collections
 * Survey123 Forms
 * Workforce Projects
 * Story Maps (classic)
 * Operation Views
 * Dashboards
 * QuickCapture Projects
 * ArcGIS Notebooks 
 * Simple Types
   * Those items with a download option (see [Data files](https://developers.arcgis.com/rest/users-groups-and-items/items-and-item-types.htm) for items that may be in a Web GIS and available for download), including zipped file geodatabases and shapefiles, code samples, zip files, and packages amongst others.

`clone_items()` clones the dependencies for the more complex items listed above. For example, cloning an existing web application, clones the web map and all hosted feature layers referenced in the map.

`clone_items()` will not clone map services and image services. Since these services can be published to servers other than the hosted server in a configuration, it's impossible for the function to determine where to publish them in the target. As a result, these items will copy over, but will continue to point back to the original source URL. 

In [ ]:
source = GIS(profile="your_online_profile")
print(source)

In [ ]:
target = GIS(profile="your_online_profile")
print(target)

In [ ]:
osm_flyr = source.content.get("c0a74e5332d443b299d804ee3cbd3cd3")
osm_flyr

In [ ]:
clonedItems = target.content.clone_items(items=[osm_flyr])

In [ ]:
clonedItems[0]